# bitcoin.API.ipynb
This notebook demonstrates core usage of the native Snowflake API using the Python connector.
It does not include your full use case — just setup, authentication, and basic operations.
It includes how to authenticate, create a table, and insert/query records in Snowflake.

In [1]:
# Import dependencies
import os
import logging
import snowflake.connector
from dotenv import load_dotenv
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Load environment variables
These contain sensitive credentials like username, password, account, etc.


In [2]:
# %%
load_dotenv()

SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_SCHEMA = os.getenv("SNOWFLAKE_SCHEMA")


## Connect to Snowflake

In [3]:
def connect_to_snowflake():
    conn = snowflake.connector.connect(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE,
        schema=SNOWFLAKE_SCHEMA
    )
    logger.info("✅ Connected to Snowflake")
    return conn

conn = connect_to_snowflake()


INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.4, Python Version: 3.6.4, Platform: Windows-10-10.0.26100-SP0
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO:snowflake.connector.connection:Setting use_openssl_only mode to False
INFO:__main__:✅ Connected to Snowflake


## Create a Table
# This table will be used to store time-stamped Bitcoin price values.


In [4]:
def create_btc_table(conn):
    query = """
    CREATE TABLE IF NOT EXISTS BTC_PRICES (
        timestamp TIMESTAMP,
        price_usd FLOAT
    );
    """
    conn.cursor().execute(query)
    logger.info("✅ Table BTC_PRICES created.")

create_btc_table(conn)

INFO:snowflake.connector.cursor:query: [CREATE TABLE IF NOT EXISTS BTC_PRICES ( timestamp TIMESTAMP, price_usd FLOAT );]
INFO:snowflake.connector.cursor:query execution done
INFO:__main__:✅ Table BTC_PRICES created.



## Insert a Sample Record

In [5]:
def insert_sample_price(conn):
    query = """
    INSERT INTO BTC_PRICES (timestamp, price_usd)
    VALUES (%s, %s);
    """
    conn.cursor().execute(query, (datetime.utcnow(), 99999.99))
    logger.info("📥 Sample row inserted.")

insert_sample_price(conn)

INFO:snowflake.connector.cursor:query: [INSERT INTO BTC_PRICES (timestamp, price_usd) VALUES ('2025-05-17 15:04:53.34964...]
INFO:snowflake.connector.cursor:query execution done
INFO:__main__:📥 Sample row inserted.


## Query the Table

In [6]:
def query_btc_prices(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM BTC_PRICES ORDER BY timestamp DESC LIMIT 5;")
    results = cursor.fetchall()
    for row in results:
        print(row)

query_btc_prices(conn)

INFO:snowflake.connector.cursor:query: [SELECT * FROM BTC_PRICES ORDER BY timestamp DESC LIMIT 5;]
INFO:snowflake.connector.cursor:query execution done


(datetime.datetime(2025, 5, 17, 15, 4, 53, 349640), 99999.99)
(datetime.datetime(2025, 5, 17, 1, 38, 16, 719906), 102933.0)
(datetime.datetime(2025, 5, 17, 1, 38, 5, 885555), 102936.0)
(datetime.datetime(2025, 5, 17, 1, 37, 55, 173624), 102936.0)
(datetime.datetime(2025, 5, 17, 1, 37, 44, 402821), 102936.0)



# This notebook demonstrates basic usage of Snowflake via Python: connect, create table, insert, and query.
The full project implementation is available in `bitcoin.example.ipynb`.